In [1]:
from IPython.core.display import HTML
with open('./style.css') as f:
    css = f.read()
HTML(css)

<h1>Play chess endgames using endgame databases</h1>

<h2>What should be created?</h2>
<div style="text-align: justify">
The following Jupyter Notebook is used to create the class ChessEndgame, which allows to play chess endgames. To simulate the chess game itself the Python package <a href='https://python-chess.readthedocs.io/en/latest/'>python-chess</a> is used. The endgame positions to be played can be created manually or generated randomly. White starts to move. The white moves can be executed by the user or randomly selected. The black moves are executed by the program. It relies on the Gaviota tablebase to determine the best possible move in any given case. Each game is exported and may be re-imported to retrace the game.
</div>

<h2>What is Gaviota?</h2>
<div style="text-align: justify">
Gaviota is a chess engine using <a href='https://www.chessprogramming.org/Gaviota_Tablebases'>Gaviota tablebases</a> which was developed by Miguel A. Ballicora. The tablebases are probed using a C-based API to retrieve depth-to-mate information (e.g. the amount of half-moves until a player is mated) for a given situation on a chess board. It supports endgames with up to 5 figures. However, this implementation is only capable of playing situations with 4 figures. 
</div>

<h2>Imports and custom exceptions</h2>
<div style="text-align: justify">
First of all, all the required packages must be imported. Additionally we define two exceptions to catch checkmate or stalemate in the course of the game and one exception for the singleton pattern, which is explained in more detail below.
</div>

In [63]:
import chess                                       # Simulate the chess game
import chess.gaviota                               # Load Gaviota Tablebase
from IPython.display import display, clear_output  # Better visualization and display of the chess board
import random                                      # Random moves and random creation of endgame positions
from datetime import datetime                      # Assign exported games with exact date
from pytz import timezone                          # Assign exported games with exact date
from typing import Union, List, Set                # Types to enable direct method signatures

In [3]:
class Checkmate(Exception): pass
class Stalemate(Exception): pass
class Singleton(Exception): pass

<h2>Definition of the basic construct of the class</h2>
<div style="text-align: justify">
The main class ChessEndgame, will be used to play a game. This initially consists of three methods and is later extended incrementally by further methods. The basic idea is that the class is instantiated and we can play various endgames with the help of this class. To reduce the constant opening and closing of the tablebase, the tablebase should not be opened per game run with the context manager keyword <code>with</code>, but should be managed by the instance of the class itself. To make this possible the tablebase must be opened when the instance is created and closed when the instance is deleted. 
</br></br>
However, it should be noted that in this way possible errors can occur with multiple instantiation and the resulting parallel access to the tablebase directory. For this reason, the class is implemented according to the singleton pattern, so that only one instance of the class can exist at a time. In principle, the singleton pattern works in such a way that when the class is instantiated, it is first checked whether the class has already been instantiated. If this is the case, the already existing instance will be returned.
</br></br>
<b><code>__new__(cls, tablebase_dir: str = './gaviota'):</code></b></br>
This pattern is implemented in the <code>__new__</code> method. This is a built-in dunder (double underscore) method, which is called before the actual initialization via <code>__init__</code> method. Within this method we first check with <code>hasattr(cls, 'instance')</code> whether the class (i.e. ChessEndgame) is not yet instantiated. If it is, we create an instance by calling the super constructor (in this case the constructor of object, since every class inherits implicitly from object). After that we create both the tablebase directory, which should be given as a parameter when instantiating (by default <i>./gaviota</i>) and the open tablebase as private attributes of the instance, since these should not be modified directly to avoid any errors with the open tablebase. Finally, the generated instance is returned. If an instance of the class already exists, we do not create a new one, but simply return the existing one. For usability, we also check if a different directory is specified for the tablebase when creating a new instance than the one that already exists and raise an exception to inform the user that <code>load_tablebase()</code> should be used to modify the tablebase. Without this additional check, the already existing instance would simply be returned with the old tablebase open without the user noticing. Since we have already specified the instance's attributes in the <code>__new__</code> method, we can simply use the default initialization method, so we don't need to implement <code>__init__</code> ourselves.
</br></br>
<b><code>__del__(self):</code></b></br> 
To ensure the clean closing of the tablebase access we also need to define the dunder method <code>__del__</code>. This will be called as soon as the instance is deleted. This can be done either via the <code>del</code> keyword or via the garbage collector. The latter deletes the instance if no reference to it exists anymore.
</br></br>
<b><code>load_tablebase(self, tablebase_dir: str):</code></b></br> 
Finally the method <code>load_tablebase(tablebase_dir: str)</code> is needed to change the opened tablebase. This receives a string, which must contain a directory of a tablebase. First, the currently open tablebase is closed. Then the new tablebase is opened and both the directory and the tablebase itself are stored in the private attribute <code>_tablebase_dir</code> and <code>_tablebase</code>. 
</br></br>
<b>Remark:</b> Direct data encapsulation does not exist in Python. So the two attributes <code>_tablebase_dir</code> and <code>_tablebase</code> can be retrieved directly from the instance of the class and modified via <code>instance._tablebase</code>. By convention, however, attributes beginning with an underscore are considered private and should not be used directly. Alternatively, the attributes could have been made to start with a dunder (double underscore). This way, Python name mangling intervenes and the attributes are textually replaced with <code>_classname__attribute</code>. Thus, for example, the tablebase could no longer be called by <code>instance.__tablebase</code>, but only by <code>instance._EndgamePlayer__tablebase</code>. However, this name mangling is mainly to prevent name collisions with subclasses and would only complicate the subsequent incremental creation of class methods. For this reason, data encapsulation by convention is sufficient in our case.
</div>

In [4]:
class ChessEndgame:

    def __new__(cls, tablebase_dir: str = './gaviota'):
        if not hasattr(cls, 'instance'):
            cls.instance = super(ChessEndgame, cls).__new__(cls)
            cls.instance._tablebase_dir = tablebase_dir
            cls.instance._tablebase = chess.gaviota.open_tablebase(tablebase_dir)
        elif not cls.instance._tablebase_dir == tablebase_dir:
            raise Singleton("Instance of ChessEndgame already exists with different tablebase_dir. Use load_tablebase() to change tablebase.")
        return cls.instance

    def __del__(self):
        self._tablebase.close()

    def load_tablebase(self, tablebase_dir: str):
        self._tablebase_dir = tablebase_dir
        self._tablebase.close()
        self._tablebase = chess.gaviota.open_tablebase(tablebase_dir)

<h2>Adding functionallity to the class</h2>
<div style="text-align: justify">
Now that we have the basic construct of the class that manages the tablebase, we can incrementally add methods to create the functionality of the class. The following two basic functionalities are needed: Playing endgames and importing games that have already been played.
</div>

<h3>Playing a game</h3>
<div style="text-align: justify">

<b><code>play_game(self, inp: Union[str, List[chess.Piece], None] = None, export_only: bool = False) -> None</code></b>

We need a method <code>play_game()</code> which starts the game flow and exports a log file to allow the game to be tracked and reimported if necessary. For the initalization there should be several possibilities: 
<ul>
    <li>You should be able to pass a FEN string to load an exact endgame position: <code>inp as String</code></li>
    <li>You should be able to pass a list of chess pieces to load a random endgame position containing the chess pieces: <code>inp as List</code></li>
    <li>You should be able to pass no parameter to create a completely random endgame: <code>no inp</code></li>
    <li>It should be possible to control via a parameter whether the run should be displayed or played automatically and only exported: <code>export_only as Boolean</code></li>
</ul>
 
 If the game is played and displayed normally, the player should be asked before each white move which move should be executed. Either a move in UCI format (e.g. a1a2) can be entered or the field can be left empty to execute a random move. As already mentioned, the black moves are executed automatically based on the tablebase. The programm then runs until the board is checkmate or stalemate.
</div>

In [6]:
def play_game(self, inp: Union[str, List[chess.Piece], None] = None, export_only: bool = False) -> None:

    #Unique name of the export file
    export_file = f'games/{datetime.now(timezone("Europe/Berlin")).strftime("%d-%m-%Y_%H:%M:%S:%f")}.txt'

    with open(export_file, 'w') as export:

        #Initialize the board with a FEN string, randomly with list of pieces or completely random with three pieces
        if isinstance(inp, str): 
            board = chess.Board(inp)
        elif isinstance(inp, list): 
            board = chess.Board(self._generate_fen(inp))
        else: 
            board = chess.Board(self._generate_random_fen())

        #Write starting FEN to export file
        export.write(f'{board.fen()}, ')

        try:  
            self._check_for_mates(board)                  #Checks if starting position is check- or stalemate

            while(True):                                  #Main loop for the game flow, which is interrupted only by exceptions
            
                if not export_only: display(board)        #Displays board on start or after black move
                self._white_move(board, export_only)      #Asks user for legal move, random move or autoplay and executes it on the board
                export.write(f'{board.peek()}, ')         #Writes last move (white_move) to export file
                self._check_for_mates(board)              #Checks if new board is check- or stalemate

                if not export_only: clear_output()        #Resets output

                if not export_only: display(board)        #Displays board after white move
                self._black_move(board)                   #Executes the best legal move for black on the board
                export.write(f'{board.peek()}, ')         #Writes last move (black move) to export file
                self._check_for_mates(board)              #Checks if new board is check- or stalemate

        #Excepts Checkmate Exception and ends playthrough
        except(Checkmate):
            export.write('Checkmate')
            if not export_only: 
                display(board)
                print("Checkmate")
            print(f"Saved game at {export_file}")

        #Excepts Stalemate Exception and ends playthrough
        except(Stalemate):
            export.write('Stalemate')
            if not export_only: 
                display(board)
                print("Stalemate")
            print(f"Saved game at {export_file}")

setattr(ChessEndgame, 'play_game', play_game)
del play_game

<div style="text-align: justify">

In order to increase the readabilty of the <code>play_game()</code> method, it uses several private auxiliary functions which will be explained below in further detail:
<ul>
<li><b><code>_generate_fen(self, pieces: List[chess.Piece]) -> str:</code></b></li>
<li><b><code>_generate_random_fen(self) -> str:</code></b></li>
<li><b><code>_check_for_mates(self, board: chess.Board) -> None:</code></b></li>
<li><b><code>_white_move(self, board: chess.Board, export_only: bool) -> None:</code></b></li>
<li><b><code>_black_move(self, board: chess.Board) -> None:</code></b></li>
</ul>
</div>

<div style="text-align: justify">

<b><code>_generate_fen(self, pieces: List[chess.Piece]) -> str:</code></b>

This class method will generate a FEN-string from a list of pieces, by placing the pieces randomly on the board at legal positions. It is used if no FEN-string, but a list of pieces is provided by the user to the `play_game()` method. 

After receiving a list of pieces, <code>generate_fen()</code> will initially create an empty board. Next it will define the internal method `invalid_bishops()`, which is necessary to validate the Board. The already existing method <code>.is_valid()</code> checks the board only for a certain number of criteria (see [here]("https://python-chess.readthedocs.io/en/latest/core.html?highlight=is_valid#chess.Board.is_valid")). The check whether the pieces can be in the respective position at all in the course of a legal game is only performed for the pawn (it must not be in the back row: <code>STATUS_PAWNS_ON_BACKRANK</code>). In principle, all other pieces can be in any position as a result of a legal chess game (even the knight: see [here]("https://scholarworks.sjsu.edu/cgi/viewcontent.cgi?article=8383&context=etd_theses")), but with one exception: bishops. If there are two bishops on the board, they must not be on the same square color. The internal method <code>invalid_bishops()</code> first checks if there are two bishops on the board. If so, the respective fields are calculated modulo 16, because the colors of the fields repeat every 16 fields (0, 16, 32, etc. are the same). Then we determine the exact color of the field (&lt;8 is dark (&#8793;True) if field is even, &ge;8 is dark (&#8793;True) if field is uneven). The last step is to XOR the resulting colors to determine if they are distinct.

The `generate_fen()` method will then go on to place each piece on a random field on the board until a valid board is created which is not already checkmate and finally returns the resulting FEN-string.
</div>

In [8]:
def generate_fen(self, pieces: List[chess.Piece]) -> str:

    board = chess.Board()
    board.clear()

    bishop_squares = []

    def invalid_bishops():
        if len(bishop_squares) == 2:
            square1, square2 = bishop_squares[0] % 16, bishop_squares[1] % 16
            get_color = lambda square: square%2 == 0 if square < 8 else square%2 == 1
            return not get_color(square1) ^ get_color(square2)
        return False

    while not board.is_valid() or board.is_checkmate() or invalid_bishops():

        #Get random square for each piece as a set (so we can be sure that the positions of the pieces are unique)
        squares = set()
        while not len(squares) == len(pieces):
            squares.add(random.randint(chess.A1, chess.H8))

        #Map each sqaure to a piece
        mapping = dict(zip(squares, pieces))
        
        #Get a list of bishop squares for the invalid_bishops() check
        bishop_squares = [key for (key, value) in mapping.items() if value == chess.Piece(chess.BISHOP, chess.BLACK)]

        board.set_piece_map(mapping)

    return board.fen()

setattr(ChessEndgame, '_generate_fen', generate_fen)
del generate_fen

<div style="text-align: justify">

<b><code>_generate_random_fen(self) -> str:</code></b>

This auxiliary function takes no input and will return a random FEN-string containing both kings as well as up to two additional pieces. It accomplishes this, by generating a random list of the above mentioned pieces and then calling the `generate_fen()` method to place these randomly on a board.
</div>

In [9]:
def generate_random_fen(self) -> str:
    types = [chess.ROOK, chess.KNIGHT, chess.BISHOP, chess.QUEEN, chess.PAWN, None]
    
    # Choose both kings as well as two random pieces (which may be None)
    pieces = [chess.Piece(chess.KING, chess.WHITE), 
              chess.Piece(chess.KING, chess.BLACK), 
              chess.Piece(random.choice(types), chess.BLACK), 
              chess.Piece(random.choice(types), chess.BLACK)]
    
    # Prevent two black queens
    if pieces.count(chess.QUEEN) == 2:
        pieces.remove(chess.QUEEN)
    
    # Strip out None pieces
    legal_pieces = [piece for piece in pieces if piece]
    
    return self._generate_fen(legal_pieces)

setattr(ChessEndgame, '_generate_random_fen', generate_random_fen)
del generate_random_fen

<div style="text-align: justify">

<b><code>_check_for_mates(self, board: chess.Board) -> None:</code></b>

In order to continuosly check for stalemate or checkmate, we shall use the following auxiliary function. If so, it will raise a pre-defined exception, which is caught in the main `play_game()` method.  
</div>

In [10]:
def check_for_mates(self, board: chess.Board) -> None:
    if board.is_checkmate(): raise Checkmate
    if board.is_stalemate(): raise Stalemate

setattr(ChessEndgame, '_check_for_mates', check_for_mates)
del check_for_mates

<div style="text-align: justify">

<b><code>_white_move(self, board: chess.Board, export_only: bool) -> None:</code></b>

The next function performs the white move, executed by the user of the game. It receives the current board on which it is to perform the move, as well as an `export_only` flag, which may be set. If the latter is set, a random white move will be selected from a list of legal moves. Otherwise, the game will enter a prompt mode, allowing the user to insert the next white move using UCI format. An empty input will also cause a random legal move to be performed. If the entered move is not valid, or does not resemble valid UCI format, corresponding error messages will be displayed. 
</div>

In [11]:
def white_move(self, board: chess.Board, export_only: bool) -> None:

    if export_only:
        board.push(random.choice(list(board.generate_legal_moves())))
        return

    while True:
        print("Please insert your move in UCI format (e.g. a1a2) or leave blank to play a random move:")
        move_uci = input()
        if move_uci == '':
           board.push(random.choice(list(board.generate_legal_moves())))
           return
        try:
            move = chess.Move.from_uci(move_uci)
            if move in board.generate_legal_moves():
                board.push(move)
                return
            print(f"{move_uci} is not a valid move.")
        except ValueError:
            print(f"{move_uci} is not in valid UCI format.")

setattr(ChessEndgame, '_white_move', white_move)
del white_move

<div style="text-align: justify">

<b><code>_black_move(self, board: chess.Board) -> None:</code></b>

The `black_move()` method executes the black move, as calculated using the Gaviota tablebase. It receives only the current board object as an argument, on which it will perform the next move. 

First, we create a generator object, containing the available legal moves. The first move will be performed, in order to check for a checkmate. If we did not reach checkmate yet, the first performed move will be stored as the currently best move in the `best_move` variable. Combined with the first move in a tuple, we will store the depth-to-mate value (dtm), which is retrieved from the tablebase. The last move will then be reverted.

We will then enter a loop over all available legal moves. Each move will be performed on the board. After checking for a checkmate, we will probe the tablebase for the current dtm. The value will then be compared to the dtm of the current best move using the `is_better_move()` method. If the method determines the new move to be better than the `best_move`, it will overwrite the `best_move` with the values of the current move. Lastly, the current move is reverted on the board, in order to prepare for the next possible move in the loop.

After the loop, the best of all legal moves is stored together with its dtm in the `best_move` variable. This is the move which is finally performed on the board, before returning. 
</div>

In [12]:
def black_move(self, board: chess.Board) -> None:
    legal_moves = board.generate_legal_moves()
    move = next(legal_moves)
    board.push(move)
    if board.is_checkmate(): 
        return
    best_move = (move, self._tablebase.probe_dtm(board))
    board.pop()

    for move in legal_moves:
        board.push(move)
        if board.is_checkmate(): 
            return
        dtm = self._tablebase.probe_dtm(board)
        if self._is_better_move(dtm, best_move[1]):
            best_move = (move, dtm)
        board.pop()

    board.push(best_move[0])
    print(f"Current dtm: {best_move[1]}")
    return

setattr(ChessEndgame, '_black_move', black_move)
del black_move

<div style="text-align: justify">

The method above uses a further auxiliary function `_is_better_move()` in order to determine the best of all possible moves:

<b><code>_is_better_move(self, new: int, old: int) -> bool</code></b>

It takes two integers, which resemble two depth-to-mate values (as determinded by the Gaviota tablebase) of two different black moves, which are compared. The method will return True, if the move corresponding to the "new" argument is better than the "old" move. 

Due to the specification of the depth-to-mate, the algorithm had to be implemented manually:
<ul>
<li>If the value is positive, the side to move is winning, otherwise it is negative</li>
<li>The absolute value is the number of half-moves until forced mate</li>
<li>If the value is zero, the game will lead to a draw or is checkmate (this is why we have to check for checkmate before comparing moves)</li>
</ul>

This means we have three cases:
<ol>
<li> 
    If the old value is positive, white is winning. Better moves would be:
        <ul>
        <li>Any greater value than the current, which would increase the number of moves until forced mate</li>
        <li>Any value of zero or lower: this would flip the game, by indicating either a draw, or that black is winning</li>
        </ul>
</li>
<li>
    If the old value is negative, black is winning. Better moves would be:
        <ul>
        <li>Any value closer to zero (greater than old value), but not zero (because we already checked for checkmate) and not positive</li>
        </ul>
</li>
<li>
    If the old value is zero, the game is facing a draw. Better moves would be:
        <ul>
        <li>Any value less than zero, which would mean that black is winning</li>
        </ul>
</li>
</ol>
</div>

In [13]:
def is_better_move(self, new: int, old: int) -> bool:
    if old > 0:
        if new > 0:
            return new > old
        elif new < 0:
            return True
        else:
            return True
    elif old < 0:
        if new > 0:
            return False
        elif new < 0:
            return new > old
        else:
            return False
    else:
        if new > 0:
            return False
        elif new < 0:
            return True
        else:
            return False

setattr(ChessEndgame, '_is_better_move', is_better_move)
del is_better_move

<h3>Importing a game from file</h3>
<div style="text-align: justify">

<b><code>import_game(self, import_file: str) -> None:</code></b>

The following method `import_game()` allows a player to import a saved game and retrace the moves. It takes the filename of the saved game as a argument and displays all stored moves in order. 
</div>

In [14]:
def import_game(self, import_file: str) -> None:
    
    with open(import_file, 'r') as game:
        
        steps = game.readline().rstrip().split(', ')

        if steps[-1] not in ['Checkmate', 'Stalemate']:
            print('Unfinished game file')
            return

        board = chess.Board(steps[0])
        display(board)

        for step in steps[1:]:
            if step in ['Checkmate', 'Stalemate']:
                print(step)
                return
            board.push(chess.Move.from_uci(step))
            display(board)


setattr(ChessEndgame, 'import_game', import_game)
del import_game

In [15]:
chessEndgame = ChessEndgame()

In [17]:
chessEndgame.play_game()

In [ ]:
chessEndgame.import_game("games/21-01-2022_20:18:10:252010.txt")

In [49]:
with open("./tables/Kkr.txt", "r") as f:
    sets = f.readlines()
    for s in sets:
        exec(s)

In [65]:
def test_dtm_with_gaviota(self, s: Set[str], dtm: int):
    for epd in s:
        board = chess.Board(epd)
        gaviota_dtm = abs(self._tablebase.probe_dtm(board))
        assert gaviota_dtm == dtm, f"FAILED: {board.epd()} -> dtm ({dtm}) != gaviota_dtm ({gaviota_dtm})"
setattr(ChessEndgame, 'test_dtm_with_gaviota', test_dtm_with_gaviota)
del test_dtm_with_gaviota

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=f6e34dfb-c85e-40db-bde6-d0ca8b0148c0' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>